In [ ]:
from pyspark.sql import SparkSession
from docx import Document
import io

# Create Spark session
spark = SparkSession.builder.appName("LargeDocxProcessing").getOrCreate()

# Define input and output paths
input_path = "/home/axat/personal/pySpark/data/file-sample_1MB_updated_inline.docx"
output_path = "/home/axat/personal/pySpark/output/final/outputfinalgb222.txt"

# Read the DOCX file as RDD
rdd = spark.sparkContext.binaryFiles(input_path)

# Define processing function
def process_docx(file_data):
    # Get the file path and content
    file_path, content = file_data
    document = Document(io.BytesIO(content))

    text = ""
    for paragraph in document.paragraphs:
        text += paragraph.text + "\n"
    return text

# Apply processing function to each DOCX file
processed_text_rdd = rdd.map(process_docx)

combined_text = processed_text_rdd.collect()

# Repartition to a single partition and save as a text file
with open(output_path, 'w') as file:
    file.write('\n'.join(combined_text))


# Stop the Spark session
spark.stop()
